# Visualize the classifier results

In [ ]:
# caching of feature vectors

%run scripts/feature_vector.py
import pickle

#class1_features = get_indexed_feature_vectors_from_directory('logos/')
#class2_features = get_indexed_feature_vectors_from_directory('non-logos/')

#with open('cache/indexed-logo-features.pickle', "w") as fp:
#    pickle.dump(class1_features, fp)
#with open('cache/indexed-non-logo-features.pickle', "w") as fp:
#    pickle.dump(class2_features, fp)



In [ ]:
%%html
<style>
.res {float:left;margin:0 0 5px 10px!important;border:5px solid red}
.cl1 {border-color:#ccc}
</style>

## Define helper functions

In [ ]:
from IPython.core.display import display, HTML
from __future__ import print_function
from __future__ import division

def get_validation_result(classifier, directory, pickle_file, expected_result):
    '''  
        read feature vectors from cache
        and predict them with the provided classifier
    '''
    testset = get_pickled_file('cache/%s.pickle' % pickle_file)
    counter = {'good': 0, 'bad': 0, 'bad_files': []}
    for root, _, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            feature_vector = testset[file_path]
            result      = classifier.predict([feature_vector])
            good = (result[0] == expected_result)
            counter['good' if good else 'bad'] += 1
            if not good: 
                counter['bad_files'].append(file_path)
    return counter

def print_images(images):
    string = ''
    for file_path in images:
        string += '<div style="float:left"><img class="res" src="' + file_path + '" width="100px" /></div>'
    display(HTML(string))

def print_validation_result(result):
    percentage=result['bad']/result['good']*100
    print("%d correctly classified files (%0.2f%%)\n%d wrongly classified files (%0.2f%%):" % (result['good'], 100-percentage, result['bad'], percentage))


## Display the results of the classifier test

In [ ]:
from __future__ import division
from __future__ import print_function
classifier_file = 'classifiers/classifier2-logos-all-static-training-set.pickle'
#classifier_file = 'classifiers/classifier-logos-static-training-set.pickle'
#classifier_file = 'classifiers/classifier3-logos-static-training-set.pickle'

classifier = pickle.load(open(classifier_file, "r"))
result1 = get_validation_result(classifier, 'logos/', 'indexed-logo-features' ,1)
result2 = get_validation_result(classifier, 'non-logos/', 'indexed-non-logo-features' ,0)

print("Classifier: \n%s\n" % classifier_file)
print('Total results:')
result = {'good': result1['good'] + result2['good'],
        'bad': result1['bad'] + result2['bad']}
print_validation_result(result)

print("\nResults: Logos")
print_validation_result(result1)
print_images(result1['bad_files'])

print("Results: Non-logos")
print_validation_result(result2)
print_images(result2['bad_files'])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def draw_pie(plt, correct, wrong):
    plt.pie((correct, wrong),
        explode=(0,0.1),
        #labels=(), # 'correct','wrong'
        colors=('gold','lightcoral'),
        autopct='%1.1f%%',
        shadow=True,
        textprops={}
       )

    plt.axis('equal')


fig =  plt.figure(figsize=(16, 6)) 
ax = fig.add_subplot(1,3,1)
draw_pie(ax, result['good'], result['bad'])
ax = fig.add_subplot(1,3,2)
draw_pie(ax, result1['good'], result1['bad'])
ax = fig.add_subplot(1,3,3)
draw_pie(ax, result2['good'], result2['bad'])

print("results using the images that were used for training\n")
print("total \t\t\t\tlogos\t\t\t\tnon-logos")
plt.show()
